In [1]:
import pandas as pd

from pendulum import now

In [2]:
dataset_name = 'sidra_tab5457_prod_agricola'
raiz = '../database'

### Bronze

Quantidade produzida (t)

In [3]:
culturas = [40129, 40092, 45982, 40130, 40100, 40101, 40102, 40103, 40131, 40136, 40104, 40105, 40137, 40138, 40139, 40106, 40142, 40143, 40107,
         40108, 40109, 40144, 40145, 40146, 40147, 40110, 40111, 40112, 40148, 40113, 40114, 40149, 40150, 40115, 40151, 40152, 40116, 40260,
         40117, 40261, 40118, 40119, 40262, 40263, 40264, 40120, 40121, 40122, 40265, 40266, 40267, 40268, 40269, 40123, 40270, 40124, 40125,
         40271, 40126, 40127, 40128, 40272, 40273, 40274]

contador = 1

ultimo_ano = now().year - 1

In [4]:
for ano in range(2000, ultimo_ano):
    for cultura in culturas:
        if contador == 1:
            df_5457 = pd.read_json(f'https://apisidra.ibge.gov.br/values/t/5457/n6/all/v/214/p/{ano}/c782/{cultura}')
            contador += 1
        else:
            df_5457 = pd.concat(
                [df_5457,
                 (pd.read_json(f'https://apisidra.ibge.gov.br/values/t/5457/n6/all/v/214/p/{ano}/c782/{cultura}')
                  .iloc[1:])],
                  ignore_index=True
                  )
            contador += 1

df_5457.to_json(f'{raiz}/bronze_{dataset_name}.json')

print(f'Foram contabilizadas {contador} repetições de produções agrícolas por anos para serem calculadas.')

Foram contabilizadas 1409 repetições de produções agrícolas por anos para serem calculadas.


### Prata

Os dados relativo ao ano 2000 são expressos em mil frutos (mil cachos no caso da banana). Para converter para toneladas foi utilizada a relação de rendimento médio nacional em toneladas/ha de 2001 com o obtido no ano 2000 em frutos/ha. Esta relação é feita considerando que não houveram grandes mudanças de rendimento da produção entre 2000 e 2001, assim sendo estimado o peso médio dos frutos. Necessitam desta estimativa as seguintes produções agrícolas:

| Produção | Produção |
|---| --- | 
| Abacate | Mamão |
| Banana | Manga |
| Caqui | Maracujá |
| Figo | Marmelo | 
| Goiaba | Melancia |
| Laranja | Melão |
| Limão | Pera | 
| Maçã |Pêssego |
| Tangerina | 

As produções de **Coco** e de **Abacaxi** são informadas em nº de mil frutos pelo IBGE. Para obtenção do peso médio do fruto, foi comparado o valor de rendimento por área do FAOSTAT (em peso por ha) com o rendimento do IBGE em frutos/ha. Com a compatibilização destes foi possível obter os valores da produção em toneladas/ha. Coincidentemente o valor de peso médio encontrado foi 1,5 kg para todos os anos em ambas as culturas.


_Base de cálculo: (Valor * 1000) * (Peso médio/1000)_

In [5]:
peso_medio = {
    40129: {'pm': 0.38}, # Abacate
    40092: {'pm': 1.5}, # Abacaxi
    40136: {'pm': 10.20}, # Banana
    40142: {'pm': 0.18}, # Caqui
    40145: {'pm': 1.5}, # Coco
    40148: {'pm': 0.09}, # Figo
    40149: {'pm': 0.16}, # Goiaba
    40151: {'pm': 0.16}, # Laranja
    40152: {'pm': 0.10}, # Limão
    40260: {'pm': 0.15}, # Maçã
    40261: {'pm': 0.80}, # Mamão
    40262: {'pm': 0.31}, # Manga
    40263: {'pm': 0.15}, # Maracujá
    40264: {'pm': 0.19}, # Marmelo
    40267: {'pm': 0.17}, # Pera
    40268: {'pm': 0.13}, # Pêssego
    40271: {'pm': 0.15}, # Tangerina
    40120: {'pm': 6.08}, # Melancia
    40121: {'pm': 1.39} # Melão
    }

def toneladas(linha):
    if linha['cultura'] == 40092 or linha['cultura'] == 40145:
        return (linha['producao_t'] * 1000) * (peso_medio[linha['cultura']]['pm']/1000)
    if linha['ano'] == 2000 and linha['cultura'] in (40129, 40136, 40142, 40148, 40149, 40151, 40152, 40260, 40261, 40262, 40263, 40264, 40267, 40268, 40271, 40120, 40121):
        return (linha['producao_t'] * 1000) * (peso_medio[linha['cultura']]['pm']/1000)
    return linha['producao_t']

In [6]:
(pd.read_json(f'{raiz}/bronze_{dataset_name}.json')
 .drop(0, axis = 0)
 .filter(['D1C', 'D3C', 'D4C', 'V'])
 .assign(**{
        'uf': lambda df: df['D1C'].str.slice(0,2).astype(int),
        'producao_t': lambda df: pd.to_numeric(df['V'], errors='coerce'),
        'municipio': lambda df: df['D1C'].astype(int),
        'ano': lambda df: df['D3C'].astype(int),
        'cultura': lambda df: df['D4C'].astype(int),
        'fonte': 'Produção Agrícola'
        })
 .dropna(subset='producao_t')
 .assign(producao = lambda x: x.apply(toneladas, axis = 1))
 .convert_dtypes()
 .groupby(['uf', 'municipio', 'ano', 'cultura', 'fonte'])
 .agg({'producao_t':sum})
 .to_parquet(f'{raiz}/prata_{dataset_name}.parquet')
)

### Ouro

| Código | Produção Agrícola | Teor N | Teor P | Teor K |
| ------ | ------------------ | ------ | ------ | ------ |
| 40129  | Abacate            | 2,2    | 0,3    | 3,7    |
| 40092  | Abacaxi*           | 0,8    | 0,2    | 2,6    |
| 45982  | Açaí [2015 a 2022] | 13     | 0,5    | 9      |
| 40130  | Algodão arbóreo (em caroço) | 22 | 3      | 15     |
| 40100  | Alho               | 7,3    | 1,1    | 4,7    |
| 40101  | Amendoim (em casca) | 35    | 2,4    | 7,1    |
| 40102  | Arroz (em casca)   | 13     | 2,9    | 3      |
| 40103  | Aveia (em grão)    | 24     | 3,8    | 4,9    |
| 40131  | Azeitona           | 7,2    | 1,1    | 10,9   |
| 40136  | Banana (cacho)     | 1,9    | 0,3    | 8,2    |
| 40104  | Batata-doce        | 5,2    | 1      | 8,3    |
| 40105  | Batata-inglesa     | 3      | 0,7    | 5,4    |
| 40137  | Borracha (látex coagulado) [1981 a 2022] | 6 | 5 | 3 |
| 40138  | Cacau (em amêndoa) | 33     | 2      | 8      |
| 40139  | Café (em grão) Total | 17,5 | 1,2    | 26,5   |
| 40106  | Cana-de-açúcar     | 1      | 0,3    | 1,5    |
| 40142  | Caqui              | 0,9    | 0,2    | 1,9    |
| 40143  | Castanha de caju [1988 a 2022] | 13,8 | 2 | 6,5 |
| 40107  | Cebola             | 2,7    | 0,5    | 2,7    |
| 40108  | Centeio (em grão)  | 25     | 3,6    | 4,6    |
| 40109  | Cevada (em grão)   | 21     | 3,6    | 5,6    |
| 40144  | Chá-da-índia (folha verde) | 27 | 4 | 13   |
| 40145  | Coco-da-baía*      | 0,3    | 0      | 0,5    |
| 40146  | Dendê (cacho de coco) [1988 a 2022] | 2,6 | 0,5 | 5,9 |
| 40147  | Erva-mate (folha verde) [1981 a 2022] | 10,7 | 0,7 | 8,7 |
| 40110  | Ervilha (em grão) [1988 a 2022] | 25 | 2,9 | 18   |
| 40111  | Fava (em grão)     | 3      | 0,4    | 1,2    |
| 40112  | Feijão (em grão)   | 50     | 5,7    | 12,5   |
| 40148  | Figo               | 3,1    | 0,5    | 0,3    |
| 40113  | Fumo (em folha)    | 36     | 3,9    | 47,3   |
| 40114  | Girassol (em grão) [2005 a 2022] | 27 | 4,2 | 7,5 |
| 40149  | Goiaba [1988 a 2022] | 1     | 0,2    | 1,6    |
| 40150  | Guaraná (semente) [1981 a 2022] | 72,3 | 11 | 56,9 |
| 40115  | Juta (fibra)       | 9,3    | 2,8    | 27     |
| 40151  | Laranja            | 4,4    | 0,4    | 4,6    |
| 40152  | Limão              | 3,3    | 0,4    | 2,9    |
| 40116  | Linho (semente)    | 13     | 1,3    | 32,4   |
| 40260  | Maçã               | 3      | 0,8    | 7      |
| 40117  | Malva (fibra)      | 33,9   | 7,6    | 48,5   |
| 40261  | Mamão              | 1,8    | 0,5    | 2,1    |
| 40118  | Mamona (baga)      | 29     | 3,7    | 7,2    |
| 40119  | Mandioca           | 2,1    | 0,2    | 2      |
| 40262  | Manga              | 1      | 0,2    | 1,6    |
| 40263  | Maracujá [1988 a 2022] | 4     | 0,7    | 3,5    |
| 40264  | Marmelo            | 5,8    | 1      | 7,5    |
| 40120  | Melancia           | 1,5    | 0,2    | 1,1    |
| 40121  | Melão              | 1,9    | 0,5    | 2,3    |
| 40122  | Milho (em grão)    | 12     | 2,7    | 3,7    |
| 40265  | Noz (fruto seco)   | 8      | 0,6    | 0,8    |
| 40266  | Palmito [1981 a 2022] | 4,32  | 1,4    | 18     |
| 40267  | Pera               | 2,9    | 0,4    | 2,6    |
| 40268  | Pêssego            | 3,2    | 0,6    | 3,3    |
| 40269  | Pimenta-do-reino   | 1,6    | 0,3    | 2,2    |
| 40123  | Rami (fibra)       | 33,9   | 7,6    | 48,5   |
| 40270  | Sisal ou agave (fibra) | 1,2 | 0,2    | 1,4    |
| 40124  | Soja (em grão)     | 55     | 5,2    | 16,6   |
| 40125  | Sorgo (em grão)    | 13     | 3,4    | 4,5    |
| 40271  | Tangerina          | 1,8    | 0,2    | 2,5    |
| 40126  | Tomate             | 1,3    | 0,2    | 2,4    |
| 40127  | Trigo (em grão)    | 21     | 4,1    | 4,6    |
| 40128  | Triticale (em grão) [2005 a 2022] | 21 | 3 | 6 |
| 40272  | Tungue (fruto seco) | 7,2   | 4,2    | 30,5   |
| 40273  | Urucum (semente) [1988 a 2022] | 10 | 2,4 | 17,2 |
| 40274  | Uva                | 4,2    | 0,7    | 5,4    |

Teor (kg/t)

Cálculo: (Produção * Teor N,P,K)/1000

In [7]:
parametro = {
    40129: {'n': 2.2, 'p': 0.3, 'k': 3.7}, # Abacate
    40092: {'n': 0.8, 'p': 0.2, 'k': 2.6}, # Abacaxi
    45982: {'n': 13, 'p': 0.5, 'k': 9}, # Açaí
    40130: {'n': 22, 'p': 3, 'k': 15}, # Algodão arbóreo
    40100: {'n': 7.3, 'p': 1.1, 'k': 4.7}, # Alho
    40101: {'n': 35, 'p': 2.4, 'k': 7.1}, # Amendoim
    40102: {'n': 13, 'p': 2.9, 'k': 3}, # Arroz
    40103: {'n': 24, 'p': 3.8, 'k': 4.9}, # Aveia
    40131: {'n': 7.2, 'p': 1.1, 'k': 10.9}, # Azeitona
    40136: {'n': 1.9, 'p': 0.3, 'k': 8.2}, # Banana
    40104: {'n': 5.2, 'p': 1, 'k': 8.3}, # Batata-doce
    40105: {'n': 3, 'p': 0.7, 'k': 5.4}, # Batata-inglesa
    40137: {'n': 6, 'p': 5, 'k': 3}, # Borracha
    40138: {'n': 33, 'p': 2, 'k': 8}, # Cacau
    40139: {'n': 17.5, 'p': 1.2, 'k': 26.5}, # Café
    40106: {'n': 1, 'p': 0.3, 'k': 1.5}, # Cana-de-açúcar
    40142: {'n': 0.9, 'p': 0.2, 'k': 1.9}, # Caqui
    40143: {'n': 13.8, 'p': 2, 'k': 6.5}, # Castanha de caju
    40107: {'n': 2.7, 'p': 0.5, 'k': 2.7}, # Cebola
    40108: {'n': 25, 'p': 3.6, 'k': 4.6}, # Centeio
    40109: {'n': 21, 'p': 3.6, 'k': 5.6}, # Cevada
    40144: {'n': 27, 'p': 4, 'k': 13}, # Chá-da-índia
    40145: {'n': 0.3, 'p': 0, 'k': 0.5}, # Coco-da-baía
    40146: {'n': 2.6, 'p': 0.5, 'k': 5.9}, # Dendê
    40147: {'n': 10.7, 'p': 0.7, 'k': 8.7}, # Erva-mate
    40110: {'n': 25, 'p': 2.9, 'k': 18}, # Ervilha
    40111: {'n': 3, 'p': 0.4, 'k': 1.2}, # Fava
    40112: {'n': 50, 'p': 5.7, 'k': 12.5}, # Feijão
    40148: {'n': 3.1, 'p': 0.5, 'k': 0.3}, # Figo
    40113: {'n': 36, 'p': 3.9, 'k': 47.3}, # Fumo
    40114: {'n': 27, 'p': 4.2, 'k': 7.5}, # Girassol
    40149: {'n': 1, 'p': 0.2, 'k': 1.6}, # Goiaba
    40150: {'n': 72.3, 'p': 11, 'k': 56.9}, # Guaraná
    40115: {'n': 9.3, 'p': 2.8, 'k': 27}, # Juta
    40151: {'n': 4.4, 'p': 0.4, 'k': 4.6}, # Laranja
    40152: {'n': 3.3, 'p': 0.4, 'k': 2.9}, # Limão
    40116: {'n': 13, 'p': 1.3, 'k': 32.4}, # Linho
    40260: {'n': 3, 'p': 0.8, 'k': 7}, # Maçã
    40117: {'n': 33.9, 'p': 7.6, 'k': 48.5}, # Malva
    40261: {'n': 1.8, 'p': 0.5, 'k': 2.1}, # Mamão
    40118: {'n': 29, 'p': 3.7, 'k': 7.2}, # Mamona
    40119: {'n': 2.1, 'p': 0.2, 'k': 2}, # Mandioca
    40262: {'n': 1, 'p': 0.2, 'k': 1.6}, # Manga
    40263: {'n': 4, 'p': 0.7, 'k': 3.5}, # Maracujá
    40264: {'n': 5.8, 'p': 1, 'k': 7.5}, # Marmelo
    40120: {'n': 1.5, 'p': 0.2, 'k': 1.1}, # Melancia
    40121: {'n': 1.9, 'p': 0.5, 'k': 2.3}, # Melão
    40122: {'n': 12, 'p': 2.7, 'k': 3.7}, # Milho
    40265: {'n': 8, 'p': 0.6, 'k': 0.8}, # Noz
    40266: {'n': 4.32, 'p': 1.4, 'k': 18}, # Palmito
    40267: {'n': 2.9, 'p': 0.4, 'k': 2.6}, # Pera
    40268: {'n': 3.2, 'p': 0.6, 'k': 3.3}, # Pêssego
    40269: {'n': 1.6, 'p': 0.3, 'k': 2.2}, # Pimenta-do-reino
    40123: {'n': 33.9, 'p': 7.6, 'k': 48.5}, # Rami
    40270: {'n': 1.2, 'p': 0.2, 'k': 1.4}, # Sisal ou agave
    40124: {'n': 55, 'p': 5.2, 'k': 16.6}, # Soja
    40125: {'n': 13, 'p': 3.4, 'k': 4.5}, # Sorgo
    40271: {'n': 1.8, 'p': 0.2, 'k': 2.5}, # Tangerina
    40126: {'n': 1.3, 'p': 0.2, 'k': 2.4}, # Tomate
    40127: {'n': 21, 'p': 4.1, 'k': 4.6}, # Trigo
    40128: {'n': 21, 'p': 3, 'k': 6}, # Triticale
    40272: {'n': 7.2, 'p': 4.2, 'k': 30.5}, # Tungue
    40273: {'n': 10, 'p': 2.4, 'k': 17.2}, # Urucum
    40274: {'n': 4.2, 'p': 0.7, 'k': 5.4} # Uva
    }


In [8]:
df = (pd.read_parquet(f'{raiz}/prata_{dataset_name}.parquet')
      .groupby(['uf', 'ano', 'cultura', 'fonte'])
      .agg({'producao_t':sum})
      .reset_index()
      .assign(**{
          'N': lambda df: df.apply(lambda lin: (lin['producao_t'] * parametro[lin['cultura']]['n'])/1000, axis = 1),
          'P': lambda df: df.apply(lambda lin: (lin['producao_t'] * parametro[lin['cultura']]['p'])/1000, axis = 1),
          'K': lambda df: df.apply(lambda lin: (lin['producao_t'] * parametro[lin['cultura']]['k'])/1000, axis = 1)
          })
      .drop('producao_t', axis = 1)
      .melt(id_vars=['uf', 'ano', 'cultura', 'fonte'],
            value_vars=['N', 'P', 'K'],
            var_name='elemento',
            value_name='valor')
      .convert_dtypes()
      .groupby(['uf', 'ano', 'fonte', 'elemento'])
      .agg({'valor': sum})
      .reset_index(level='elemento')
      )

df.to_csv(f'{raiz}/ouro_{dataset_name}.csv')

In [9]:
df

elemento        valor
uf ano  fonte                                  
11 2000 Produção Agrícola        K    9060.1694
        Produção Agrícola        N   14102.5996
        Produção Agrícola        P    1858.8961
   2001 Produção Agrícola        K   10756.1455
        Produção Agrícola        N   15437.1123
...                            ...          ...
53 2020 Produção Agrícola        N  25345.85366
        Produção Agrícola        P    3353.3857
   2021 Produção Agrícola        K    7322.3142
        Produção Agrícola        N  23418.98516
        Produção Agrícola        P    2867.5987

[1782 rows x 2 columns]